<a href="https://colab.research.google.com/github/Rohitha2803/Health_app/blob/main/Health_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok transformers accelerate PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import os
# Set your Hugging Face API key here
os.environ["HF_TOKEN"] =" your_token "

In [ ]:

%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from PyPDF2 import PdfReader
import torch

st.set_page_config(page_title="🩺 Health AI", layout="centered")

# Load model
@st.cache_resource
def load_model():
    model_name = "ibm-granite/granite-3.3-2b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        use_auth_token=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_model()

# Extract text from PDF
def extract_text_from_pdf(uploaded_file):
    reader = PdfReader(uploaded_file)
    text = ""
    for page in reader.pages:
        if page.extract_text():
            text += page.extract_text()
    return text

# Sidebar
st.sidebar.title("🏥 Health AI")
page = st.sidebar.radio("Choose an Option", ["Medical Report", "Home Remedies", "Medical Chatbot"])

# ---------- Medical Report Analysis ----------
if page == "Medical Report":
    st.title(" Medical Report Analyzer")
    uploaded_file = st.file_uploader("Upload a medical report (PDF)", type="pdf")

    if uploaded_file:
        report_text = extract_text_from_pdf(uploaded_file)
        st.success(" Report uploaded.")

        if st.button(" Analyze Report"):
            with st.spinner("Analyzing the medical report..."):
                prompt = f"""
Analyze this medical report and give a detailed summary in a simple way.
Also based on the report, generate the health condition.
Here is the report:
{report_text[:3500]}
"""
                inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
                outputs = model.generate(**inputs, max_new_tokens=768, do_sample=True, temperature=0.7)
                result = tokenizer.decode(outputs[0], skip_special_tokens=True)
                st.subheader("🩺  Patient Report Summary")
                st.write(result)

        st.subheader("💬 Ask a Question about the Report")
        user_query = st.text_input("Ask your question...")
        if user_query:
            with st.spinner("Thinking..."):
                qa_prompt = f"Based on this report:\n{report_text[:3000]}\n\nAnswer this question in a simple way: {user_query}"
                inputs = tokenizer(qa_prompt, return_tensors="pt", truncation=True).to(model.device)
                outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)
                answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
                st.markdown("**🤖 Answer:**")
                st.write(answer)

# ---------- Home Remedies ----------
elif page == "Home Remedies":
    st.title(" Home Remedies & Self-Care Assistant")
    symptoms = st.text_area("Enter your symptoms (e.g., fever, cough, sore throat)", height=100)

    if st.button(" Get Remedies & Self-Care Tips"):
        with st.spinner("Fetching personalized suggestions..."):
            prompt = f"""
I have the following symptoms: {symptoms}

1. Suggest at least 10 simple and natural home remedies for the symptoms.
2. Also provide at least 10 self-care and lifestyle tips to manage or relieve the symptoms.
"""
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
            outputs = model.generate(**inputs, max_new_tokens=1024, do_sample=True, temperature=0.8)
            result = tokenizer.decode(outputs[0], skip_special_tokens=True)

            if "🧘" in result:
                remedies_text, selfcare_text = result.split("🧘", 1)

                st.markdown("<h3><b>1.  Home Remedies </b></h3>", unsafe_allow_html=True)
                st.write(remedies_text.strip())

                st.markdown("<h3><b>2.  Self-Care and Lifestyle Tips</b></h3>", unsafe_allow_html=True)
                st.write(selfcare_text.strip())
            else:
                st.write(result)

# ---------- Medical Chatbot ----------
elif page == "Medical Chatbot":
    st.title(" AI Medical Chatbot")
    st.markdown("Ask any general medical question below and get an AI-powered response.")

    # Initialize chat history
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    # Chat input
    user_query = st.chat_input("Ask a medical question...")

    if user_query:
        with st.spinner("Generating response..."):
            prompt = f"Answer the following medical question.:\n\n{user_query}"
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
            outputs = model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.7)
            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Save interaction
            st.session_state.chat_history.append((user_query, answer))

    # Display chat history
    for user_msg, bot_msg in st.session_state.chat_history:
        st.chat_message("user").write(user_msg)
        st.chat_message("assistant").write(bot_msg)


Writing app.py


In [ ]:


!ngrok config add-authtoken   "your_token"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/dev/null &


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f" Streamlit App is Live at: {public_url}")


🚀 Streamlit App is Live at: NgrokTunnel: "https://9fe0-35-237-206-121.ngrok-free.app" -> "http://localhost:8501"
